In [30]:
%reload_ext autoreload
%autoreload 2

# Sandbox

In [31]:
import settings
from combolists import ComboList
from accesslevel import AccessLevel
from dotenv import load_dotenv
import warnings
from card import Card
from colaborador import Colaborador
from paciente import Paciente
from time import sleep
from helperfunctions import helper
from database import DataBase
from acompanhante import Acompanhante
from estudante import Estudante
import datetime
import requests
import json
import pyodbc

warnings.filterwarnings("ignore")
load_dotenv()
settings.init()
settings.combofields = ComboList.getComboLists()
settings.accesslevels = AccessLevel.getAccessLevels()

In [32]:
nomes = [
'ANA ALICE DOS SANTOS LIMA',
'ANGELA EDUARDA COSTA SILVA',
'ANNA BEATRIZ DE BRITO MENEZES',
'BIANCA FREIRE DE CASTRO',
'BIANCA RODRIGUES CARVALHO',
'CAROLINE PAIVA DA SILVA',
'CLEITON FERREIRA DE ARAUJO',
'DEBORA VITORIA MARIA DE SOUZA FLORENCIO',
'DELVA DE OLIVEIRA CABRAL BARBOSA',
'HEITOR BEZERRA DO NASCIMENTO',
'ISABELLE KARINE RAMOS DE LIMA',
'JESSICA MARIA DA SILVA BUARQUE',
'KAROLINA XAVIER RODRIGUES BARBOSA CRUZ',
'LAIS MARIA DA SILVA COSTA',
'LETICIA CONCEICAO DE ANDRADE',
'LETICIA GABRIELLE DE ALBUQUERQUE GUIMARAES',
'LUIZ GUILHERME SANTOS DE ALMEIDA',
'MARCOS VINICIUS JOSE CARDOSO DE MELO',
'MARIA CLARA GOMES OLIVEIRA DA SILVA',
'MARIA EDUARDA DONATO CONSTANTINO',
'MARIA EDUARDA MARQUES MACHADO DA SILVA',
'MARIA EDUARDA PEREIRA ARRUDA DA SILVA',
'MARIA EDUARDA RIBEIRO MACIEL DA SILVA',
'MARIA LAYS SILVA VIEIRA',
'MARIANA RODRIGUES BUARQUE DE VASCONCELOS',
'MARIANNE PEIXOTO NOGUEIRA',
'MARIELLY VICTORIA MENDES FERREIRA',
'MILENA FREITAS SIQUEIRA',
'NATHALIA MAYARA MENDES',
'SABRINA OLIVEIRA DA SILVA',
'SUYANE SILVESTRE DA SILVA',
'THAISLINE GONCALVES PONTES',
'THALITA HELENA MENDES DE AGUIAR',
'THAYNARA TRAVASSOS PAZ DE FREITAS'
]

In [33]:
connection = pyodbc.connect(settings.sql_server)
cursor = connection.cursor()

for nome in nomes:
  query = f"SELECT CHID from dbo.CHMAIN where FirstName = '{nome}' and CHTYPE = 8"

  cursor.execute(query)
  rows = cursor.fetchall()
  if len(rows) == 1:
    chid = rows[0]
    # remove parentesis and comma from row
    chid = str(chid).replace('(', '').replace(')', '').replace(',', '')
    sleep(0.5)
    url = f"{settings.baseUrl}/cardholders/{chid}?includeTables=Cards"
    result = requests.get(
      url,
      verify=False,
      )
    estudante = result.json()
    card = estudante["Cards"][0]
    
    card["CardStartValidityDateTime"] = "2024-09-30T00:00:00"
    card["CardEndValidityDateTime"] = "2025-03-01T00:00:00"
    card["CardState"] = 0

    estudante["AuxDte02"] = "2024-09-30T00:00:00"
    estudante["AuxDte03"] = "2024-03-01T00:00:00"
    estudante["CHStartValidityDateTime"] = "2024-09-30T00:00:00"
    estudante["CHEndValidityDateTime"] = "2025-03-01T00:00:00"
    estudante["CHState"] = 0
    update_update = requests.put(
      f"{settings.baseUrl}/cardholders",
      verify=False,
      json=estudante
    )

    update_card = requests.put(
      f"{settings.baseUrl}/cards",
      verify=False,
      json=card
    )

    print(json.dumps(estudante, indent=2))
    print(update_update.status_code)

{
  "CHID": 1096304,
  "CHType": 8,
  "FirstName": "ANA ALICE DOS SANTOS LIMA",
  "LastName": null,
  "CompanyID": null,
  "VisitorCompany": null,
  "EMail": "anaalice.lima@ufpe.br",
  "CHState": 0,
  "GDPRSignatureDateTime": null,
  "IsValidGDPRSignature": false,
  "IsUndesirable": false,
  "IsUndesirableReason1": null,
  "IsUndesirableReason2": null,
  "PartitionID": 1,
  "LastModifOnLocality": 1,
  "LastModifDateTime": "2024-09-30T21:52:09.12",
  "LastModifBy": null,
  "CHStartValidityDateTime": "2024-09-30T00:00:00",
  "CHEndValidityDateTime": "2025-03-01T00:00:00",
  "CHDownloadRequired": null,
  "TraceCH": null,
  "Trace_AlmP": 1,
  "Trace_Act": null,
  "TrustedLogin": null,
  "DefFrontCardLayout": null,
  "DefBackCardLayout": null,
  "IdNumber": "6771097",
  "MaxTransits": null,
  "MaxMeals": null,
  "IgnoreTransitsCount": true,
  "IgnoreMealsCount": false,
  "IgnoreAntiPassback": false,
  "IgnoreZoneCount": false,
  "PIN": null,
  "RequiresEscort": false,
  "CanEscort": false,


In [34]:
# import pyodbc

# connection = pyodbc.connect(settings.sql_server)

# cursor = connection.cursor()

# query = """
# SELECT 
# * 
# FROM dbo.CHMAIN as cardholder
# LEFT JOIN dbo.chcards card on card.chid = cardholder.chid and card.CardState <> 2
# LEFT JOIN dbo.CHLastTransit transito on cardholder.chid = transito.CHID
# LEFT JOIN dbo.CHAUX aux ON aux.CHID = cardholder.CHID
# WHERE cardholder.chstate = 0
# AND cardholder. 
# ORDER BY cardholder.CHID
# OFFSET 0 ROWS FETCH FIRST 10 ROWS ONLY
# """

# cursor.execute(query)

# colunas = [column[0] for column in cursor.description]

# rows = cursor.fetchall()

# resultados = [dict(zip(colunas, row)) for row in rows]

# connection.close()

# print(json.dumps(resultados, indent=4, default=str))
  

In [35]:
# connection = pyodbc.connect(settings.sql_server)

# cursor = connection.cursor()

# query = f"""
# SELECT 
# cardholder.chid,
# CASE 
#   WHEN cardholder.CHType = 1 THEN 'VISITANTE'
#   WHEN cardholder.CHType = 3 THEN 'COLABORADOR'
#   WHEN cardholder.CHType = 7 THEN 'ACOMPANHANTE'
#   WHEN cardholder.CHType = 8 THEN 'ESTUDANTE'
#   WHEN cardholder.CHType = 2 THEN 'PACIENTE'
#   ELSE 'DESCONHECIDO'
# END AS tipo,
# CASE 
#   WHEN (card.cardstate = 0 and cardholder.chstate = 0) THEN 'ATIVO'
#   ELSE 'INATIVO'
# END AS ativo,
# --case
# --  when cardholder.
# --end as tipo,
# cardholder.CHEndValidityDateTime as validade,
# cardholder.CHStartValidityDateTime as inicio, 
# transito.EventDateTime as transito
# FROM chmain as cardholder
# LEFT JOIN chcards card on card.chid = cardholder.chid and card.CardState <> 2
# LEFT JOIN CHLastTransit transito on cardholder.chid = transito.CHID
# """

# cursor.execute(query)

# colunas = [column[0] for column in cursor.description]

# rows = cursor.fetchall()

# resultados = [dict(zip(colunas, row)) for row in rows]

# resultados = list(filter(lambda x: x['ativo'] == 'ATIVO', resultados))



# acompanhantes = []
## add to acompanhantes every item of resultados where tipo is ACOMPANHANTE, and (transito is null or transito is older than 3 days ago)
# for resultado in resultados:
#     if resultado['tipo'] == 'ACOMPANHANTE':
#         if resultado['transito'] is None and (datetime.datetime.now() - resultado['inicio']).days > 3:
#             acompanhantes.append(resultado)
#         else:
#             if (datetime.datetime.now() - resultado['transito']).days > 3:
#                 acompanhantes.append(resultado)

# visitantes = []
## add to visitantes every item of resultados where tipo is VISITANTE, and (transito is null or transito is older than 3 days ago)
# for resultado in resultados:
#     if resultado['tipo'] == 'VISITANTE':
#         if resultado['transito'] is None and (datetime.datetime.now() - resultado['inicio']).days > 365:
#             visitantes.append(resultado)
#         else:
#             if (datetime.datetime.now() - resultado['transito']).days > 365:
#                 visitantes.append(resultado)


# colaborador = []
## add to colaborador every item of resultados where tipo is COLABORADOR, and (transito is null or transito is older than 3 days ago) 
# for resultado in resultados:
#     if resultado['tipo'] == 'COLABORADOR' and (datetime.datetime.now() - resultado['inicio']).days >60:
#         if resultado['transito'] is None:
#             colaborador.append(resultado)
#         else:
#             if (datetime.datetime.now() - resultado['transito']).days > 60:
#                 colaborador.append(resultado)

# estudante = []
## add to estudante every item of resultados where tipo is ESTUDANTE, and (transito is null or transito is older than 3 days ago)
# for resultado in resultados:
#     if resultado['tipo'] == 'ESTUDANTE' and (datetime.datetime.now() - resultado['inicio']).days >60:
#         if resultado['transito'] is None:
#             estudante.append(resultado)
#         else:
#             if (datetime.datetime.now() - resultado['transito']).days > 60:
#                 estudante.append(resultado)

# print(f'ACOMPANHANTES com Transito nulo ou mais velho que 3 dias: {len(acompanhantes)}')
# print(f'VISITANTES com Transito nulo ou mais velho que 1 ano: {len(visitantes)}')
# print(f'COLABORADORES com Transito nulo ou mais velho que 60 dias: {len(colaborador)}')
# print(f'ESTUDANTES com Transito nulo ou mais velho que 60 dias: {len(estudante)}')
# print(f'TOTAL: {len(acompanhantes) + len(visitantes) + len(colaborador) + len(estudante)}')

# for resultado in acompanhantes:
#     print(f"{resultado['chid']} - {resultado['inicio']} || {resultado['validade']} || {resultado['transito']}")

# queue = (acompanhantes, visitantes)

# for tipo in queue:
#     for item in tipo:
#       cardholder = requests.get(
#           f'{settings.baseUrl}/cardholders/{item["chid"]}',
#           verify=False,
#       ).json()
#       cardholder["CHState"] = 1
#       print(f'Desativando usuário: {item["chid"]} do tipo {item["tipo"]}')
#       result = requests.put(
#           f'{settings.baseUrl}/cardholders',
#           verify=False,
#           headers={"Content-Type": "application/json"},
#           data=json.dumps(cardholder, default=str),
#       )
#       sleep(0.2)
